[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cozek/OffensEval2020-code/blob/master/notebooks/Eng%20Task%20B%20-%20Ensemble%20Roberta.ipynb)

# Import Libraries

At the time of our work, we used the following library versions
- numpy 1.18.1
- pandas 1.0.1
- torch 1.2.0
- Cuda 10.0
- python 3.7.0
- sklearn 0.22.1
- tqdm 4.42.1
- nltk 3.4.5

In [1]:
!git clone https://github.com/KennethSunn/Intro-to-NLP-Final-Project-Fall2021.git

Cloning into 'Intro-to-NLP-Final-Project-Fall2021'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 57 (delta 12), reused 37 (delta 3), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [2]:
!git clone https://github.com/huggingface/transformers
!pip install /content/transformers/

Cloning into 'transformers'...
remote: Enumerating objects: 91510, done.
remote: Total 91510 (delta 0), reused 0 (delta 0), pack-reused 91510
Receiving objects: 100% (91510/91510), 75.59 MiB | 28.35 MiB/s, done.
Resolving deltas: 100% (66046/66046), done.
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 15.1 MB/s 
     |████████████████████████████████| 61 kB 588 kB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |██████████████████

In [3]:
import sys
sys.path.append('/content/Intro-to-NLP-Final-Project-Fall2021/src/')
import collections
from typing import Callable
import numpy as np
np.random.seed(42)
import pandas as pd
from tqdm import notebook
import importlib
import pprint
import nltk
import datetime
import os
from argparse import Namespace

from collections import Counter

In [4]:
import utils.general as general_utils
import utils.transformer.data as transformer_data_utils
import utils.transformer.general as transformer_general_utils
general_utils.set_seed_everywhere()

In [5]:
import logging
logging.basicConfig(level=logging.INFO) 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
torch.__version__ # we used version 1.2.0


'1.10.0+cu111'

In [7]:
# Import RAdam and Lookahead
from radam.radam import RAdam
from lookahead.optimizer import Lookahead


In [8]:
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification

In [9]:
 args = Namespace(
        #use cuda by default
        device = 'cuda' if torch.cuda.is_available() else 'cpu',
    
        #set batch size and number of epochs
        batch_size = 32,
        num_epochs = 20,
    
        #set the learning rate
        learning_rate = 0.0001,

        #location of the train, dev and test csv
        train_val_csv = '/content/Intro-to-NLP-Final-Project-Fall2021/TaskB_data/train_b.csv',
        test_csv = '/content/Intro-to-NLP-Final-Project-Fall2021/TaskB_data/test_b_tweets.tsv',
        #directory to save our models at
        directory = './models/', 
        model_name = 'roberta.pt',
     
        date = datetime.datetime.now().strftime("%a_%d_%b_%Y/"),
)

## Model save location

In [10]:
directory = args.directory + args.date
if not os.path.exists(directory):
    os.makedirs(directory)
args.directory = directory
print(args.directory)

./models/Mon_13_Dec_2021/


## Load presplit dataset portion
```
Labelled as

'UNT': 1
'TIN': 0
```

In [11]:
data_df_task_b = pd.read_csv(args.train_val_csv)

In [12]:
data_df_task_b.columns

Index(['Unnamed: 0', 'id', 'text', 'average', 'std', 'label', 'split'], dtype='object')

In [13]:
data_df_task_b

,Unnamed: 0,id,text,average,std,label,split
0,0,1186857531320213507,idc if we had sex . I will stop talking to you...,0.186318,0.220494,0,train
1,1,1157369736667238401,I like how all these guys got shit to say abou...,0.291077,0.272493,0,train
2,2,1156595765579526144,Good niggas dying in the street.. these niggas...,0.213908,0.205281,0,train
3,3,1188212803212996608,@USER Let me find out he’s a travieso ass bull...,0.201181,0.234027,0,train
4,4,1161769917995257856,maybe i'm super dumb but i do NOT understand h...,0.292791,0.245540,0,train
...,...,...,...,...,...,...,...
56687,56687,1160922286163210241,Mufuckas rather dwell in there mistery rather ...,0.559829,0.350387,1,test
56688,56688,1160419444344143877,"@USER Fuck this, I'm on blog TV with my fuckin...",0.598556,0.291054,1,test
56689,56689,1187134917202845698,@USER Don't forget to tic too that shit,0.631235,0.167492,1,test
56690,56690,1161604179850215424,i fucking suck at everything,0.846225,0.122429,1,test


In [14]:
with pd.option_context('display.max_colwidth', -1): 
    print(data_df_task_b[['text','label']].sample(5))

                                                                                                                                             text  label
3003   @USER dumb and dumber                                                                                                                       0    
15000  Saying racist shit is hilarious. I totally understand white supremacy.                                                                      0    
32163  @USER halls shadow fall is fucking shitty yalls servers r shit then expect us to pay for the event                                          0    
46532  @USER What the fuck did u use?                                                                                                              1    
14579  I Know Somebody Like That 🤦🏾‍♂️niggas honestly be talking their way out of pussy. she might give you the guts if you shut your damn mouth.  0    


## Importing the Roberta Tokeniker and Punkt sentence tokenizer

In [15]:
class RobertaPreprocessor():
    def __init__(self,transformer_tokenizer,sentence_detector):
        self.transformer_tokenizer = transformer_tokenizer
        self.sentence_detector = sentence_detector
        self.bos_token = transformer_tokenizer.bos_token
        self.sep_token = ' ' + transformer_tokenizer.sep_token + ' '
    def add_special_tokens(self, text):
        sentences = self.sentence_detector.tokenize(text)
        eos_added_text  = self.sep_token.join(sentences) 
        return self.bos_token +' '+ eos_added_text + ' ' + self.transformer_tokenizer.sep_token

In [16]:
!python -c 'import nltk; nltk.download("punkt")'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
roberta_tokenizer = tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
punkt_sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [18]:
roberta_preproc = RobertaPreprocessor(roberta_tokenizer, punkt_sentence_detector)

In [19]:
#apply the preprocessor on the exploded dataframe
data_df_task_b['text'] = data_df_task_b['text'].map(roberta_preproc.add_special_tokens)

In [20]:
with pd.option_context('display.max_colwidth', -1): 
    print(data_df_task_b[['text','label']].sample(5))

                                                                                                                              text  label
12000  <s> @USER @USER What kind of psycological harm did someone did to you when you were little to be that ugly ass bitch? </s>   0    
19154  <s> i need a fat ass hug :/ </s>                                                                                             0    
35973  <s> @USER @USER emaza was about to slap her ass </s>                                                                         0    
16     <s> if you're reading this you WILL get off your ass and do that productive thing youve been putting off... no excuses </s>  0    
17415  <s> @USER Didn't he call his wife an ugly bitch or something? </s>                                                           0    


### Here we create the dataset

In [21]:
class SimpleVectorizer():
    def __init__(self,tokenizer: Callable, max_seq_len: int):
        """
        Args:
            tokenizer (Callable): transformer tokenizer
            max_seq_len (int): Maximum sequence lenght 
        """
        self.tokenizer = tokenizer
        self._max_seq_len = max_seq_len

    def vectorize(self,text :str):
        
        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False, #already added by preproc
            max_length = self._max_seq_len,
            pad_to_max_length = True,
        )
        ids =  np.array(encoded['input_ids'], dtype=np.int64)
        attn = np.array(encoded['attention_mask'], dtype=np.int64)
        
        return ids, attn

In [22]:
class HateDataset(Dataset):
    def __init__(
        self,
        data_df: pd.DataFrame,
        tokenizer: Callable,
        max_seq_length:int = None,
    ):
        """
        Args:
            data_df (pandas.DataFrame): df containing the labels and text
            tokenizer (tokenizer module for the transformer)
        """
        self.data_df = data_df
        self.tokenizer = tokenizer

        if max_seq_length is None:
            self._max_seq_length = self._get_max_len(data_df,tokenizer)
        else:
            self._max_seq_length = max_seq_length

        self.train_df = self.data_df[self.data_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.data_df[self.data_df.split == 'val']
        self.val_size = len(self.val_df)

        self.test_df = self.data_df[self.data_df.split == 'test']
        self.test_size = len(self.test_df)
        
        self._simple_vectorizer = SimpleVectorizer(tokenizer, self._max_seq_length)
        
        self._lookup_dict = {
            'train': (self.train_df, self.train_size),
            'val': (self.val_df, self.val_size),
            'test': (self.test_df, self.test_size)
        }

        self.set_split('train')

        class_counts = data_df.label.value_counts().to_dict()
         #sorted on the basis of class label,eg, 0,1,2..
        cts = sorted([(lbl,cts) for lbl,cts in class_counts.items()], key=lambda x: x[0])
        freq = [ x[1] for x in cts ]
        # print(freq,cts)
        self.class_weights = 1.0/ torch.tensor(freq, dtype=torch.float32)
    
    
    def _get_max_len(self,data_df: pd.DataFrame, tokenizer: Callable):
        len_func = lambda x: len(self.tokenizer.encode_plus(x)['input_ids'])
        max_len = data_df.text.map(len_func).max() 
        return max_len

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        
        indices, attention_masks = self._simple_vectorizer.vectorize(row.text)

        label = row.label
        return {'x_data': indices,
                'x_attn_mask': attention_masks,
                'x_index': index,
                'y_target': label}
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

In [23]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=False, device="cpu", pinned_memory = False, n_workers = 0): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            pin_memory= pinned_memory,
                            num_workers = n_workers,
                            )
    
    for data_dict in dataloader:
        out_data_dict = {}
        out_data_dict['x_data'] = data_dict['x_data'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_attn_mask'] = data_dict['x_attn_mask'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        out_data_dict['x_index'] = data_dict['x_index']
        out_data_dict['y_target'] = data_dict['y_target'].to(
            device, non_blocking= (True if pinned_memory else False) 
        )
        yield out_data_dict

In [24]:
dataset = HateDataset(
    data_df = data_df_task_b,
    tokenizer = roberta_tokenizer
)

In [25]:
assert dataset._max_seq_length <= 1024

# Initialize the Roberta  model

In [26]:
model = RobertaForSequenceClassification.from_pretrained(
    # 'roberta-base',
    'distilroberta-base',
    num_labels=len(set(data_df_task_b.label)),
)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [27]:
model.to(args.device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [28]:
early_stopping = transformer_general_utils.EarlyStopping(patience=4)

In [29]:
!nvidia-smi

Mon Dec 13 05:17:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    34W / 250W |   1471MiB / 16280MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
# args.num_epochs = 20
args.num_epochs = 1
args.batch_size = 16 #set according to GPU capacity

In [31]:
loss_func = nn.CrossEntropyLoss()

print(f'Using LR:{args.learning_rate}')
base_optimizer = RAdam(model.parameters(), lr = args.learning_rate)
optimizer = Lookahead(optimizer = base_optimizer, k = 5, alpha=0.5 )
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer.optimizer, factor =0.1 ,mode='max',
)

Using LR:0.0001


# Begin Training

In [32]:
train_state = general_utils.make_train_state()
train_state.keys()


dict_keys(['train_preds', 'train_indexes', 'train_targets', 'train_accuracies', 'train_f1s', 'train_losses', 'val_preds', 'val_indexes', 'val_targets', 'val_accuracies', 'val_f1s', 'val_losses', 'test_preds', 'test_indexes', 'test_targets', 'test_accuracies', 'test_f1s', 'test_losses', 'batch_preds', 'batch_targets', 'batch_indexes', 'epoch_index'])

In [33]:
epoch_bar = notebook.tqdm(
    desc = 'training_routine',
    total = args.num_epochs,
    position=0,
    leave = True,
)
dataset.set_split('train')
train_bar = notebook.tqdm(
    desc = 'split=train ',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)
dataset.set_split('val')
eval_bar = notebook.tqdm(
    desc = 'split=eval',
    total=dataset.get_num_batches(args.batch_size),
    position=0,
    leave=True,
)

old_val_acc = 0
old_f1 = 0
model_state = None
for epoch_index in range(args.num_epochs):
    train_state['epoch_in'] = epoch_index

    dataset.set_split('train')

    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 3, 
    )

    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    model.train()

    train_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )

    for batch_index, batch_dict in enumerate(batch_generator):
        optimizer.zero_grad()
        
        loss,y_pred = model(
            input_ids = batch_dict['x_data'],
            attention_mask =  batch_dict['x_attn_mask'],
            labels= batch_dict['y_target'].unsqueeze(1),
        )[:2]
        
        y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
                             
        loss.backward()
        optimizer.step()
                             
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)
                             
        y_pred = y_pred.detach().cpu()
        batch_dict['y_target'] = batch_dict['y_target'].cpu()
        
        acc_t = transformer_general_utils \
            .compute_accuracy(y_pred, batch_dict['y_target'])
        
        f1_t = transformer_general_utils \
            .compute_macro_f1(y_pred, batch_dict['y_target'])

        train_state['batch_preds'].append(y_pred)
        train_state['batch_targets'].append(batch_dict['y_target'])
        train_state['batch_indexes'].append(batch_dict['x_index'])

        running_acc += (acc_t - running_acc) / (batch_index + 1)
        running_f1 += (f1_t - running_f1) / (batch_index + 1)

        train_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                             epoch=epoch_index)

        train_bar.update()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    train_state['train_accuracies'].append(running_acc)
    train_state['train_losses'].append(running_loss)
    
    train_state['train_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )
    train_state['train_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['train_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    train_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['train_preds'][-1],
                                  train_state['train_targets'][-1],
                                 )
                                 
    train_state['train_f1s'].append(train_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    
    dataset.set_split('val')
    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=True,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 2, 
    )
    eval_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = 0.0
    
    model.eval()
    with torch.no_grad():
        optimizer._backup_and_load_cache()
        for batch_index, batch_dict in enumerate(batch_generator):
            loss, y_pred = model(
                input_ids = batch_dict['x_data'],
                attention_mask =  batch_dict['x_attn_mask'],
                labels= batch_dict['y_target'].unsqueeze(1),
            )[:2]
            y_pred = y_pred.view(-1, len(set(dataset.data_df.label)))
            
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            y_pred = y_pred.detach()
            batch_dict['y_target'] = batch_dict['y_target'].cpu()
            
            acc_t = transformer_general_utils\
                .compute_accuracy(y_pred, batch_dict['y_target'])
            f1_t = transformer_general_utils \
                .compute_macro_f1(y_pred, batch_dict['y_target'])

            train_state['batch_preds'].append(y_pred.cpu())
            train_state['batch_targets'].append(batch_dict['y_target'].cpu())
            train_state['batch_indexes'].append(batch_dict['x_index'].cpu())

            running_acc += (acc_t - running_acc) / (batch_index + 1)
            running_f1 += (f1_t - running_f1) / (batch_index + 1)
            

            eval_bar.set_postfix(loss = running_loss, f1 = running_f1, acc=running_acc,
                                 epoch=epoch_index)
            eval_bar.update()
            
    train_state['val_accuracies'].append(running_acc)
    train_state['val_losses'].append(running_loss)
    
        
    train_state['val_preds'].append(
        torch.cat(train_state['batch_preds']).cpu()
    )

    train_state['val_targets'].append(
        torch.cat(train_state['batch_targets']).cpu()
    )
    train_state['val_indexes'].append(
        torch.cat(train_state['batch_indexes']).cpu()
    )
    val_f1 = transformer_general_utils \
                .compute_macro_f1(train_state['val_preds'][-1],
                                  train_state['val_targets'][-1],
                                 )
                                 
    train_state['val_f1s'].append(val_f1)
    
    train_state['batch_preds'] = []
    train_state['batch_targets'] = []
    train_state['batch_indexes'] = []
    
    torch.save(
        {
            'model':model.state_dict(),
        },
        args.directory + f'_epoc_{epoch_index}_' + args.model_name,
    )
    
    scheduler.step(val_f1)
    early_stopping(val_f1, model)
    optimizer._clear_and_load_backup()
    epoch_bar.set_postfix( best_f1 = early_stopping.best_score, current = val_f1)
    epoch_bar.update()    
    
    if early_stopping.early_stop:
        print("Early stopping")
        break


training_routine:   0%|          | 0/1 [00:00<?, ?it/s]

split=train :   0%|          | 0/3197 [00:00<?, ?it/s]

split=eval:   0%|          | 0/177 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulti

In [34]:
epoch_index

0

In [35]:
print(train_state['val_f1s'])

[0.84607272258193]


In [36]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [37]:

print('Train:',classification_report(
    y_pred=(torch.argmax(train_state['train_preds'][-1],dim=1) ).cpu().long().numpy(),
    y_true= train_state['train_targets'][-1].cpu().numpy(), 
    digits=4)
)
print('Dev:',classification_report(
    y_pred=(torch.argmax(train_state['val_preds'][-1],dim=1) ).cpu().long().numpy(),
    y_true= train_state['val_targets'][-1].cpu().numpy(), 
    digits=4)
)


Train:               precision    recall  f1-score   support

           0     0.8956    0.9442    0.9193     40491
           1     0.7335    0.5823    0.6492     10674

    accuracy                         0.8687     51165
   macro avg     0.8145    0.7632    0.7842     51165
weighted avg     0.8617    0.8687    0.8629     51165

Dev:               precision    recall  f1-score   support

           0     0.9358    0.9358    0.9358      2243
           1     0.7563    0.7563    0.7563       591

    accuracy                         0.8984      2834
   macro avg     0.8461    0.8461    0.8461      2834
weighted avg     0.8984    0.8984    0.8984      2834



In [38]:
best_run_index = train_state['val_f1s'].index(max(train_state['val_f1s']))
print('Train:',classification_report(
    y_pred=(torch.argmax(train_state['train_preds'][best_run_index],dim=1) ).cpu().long().numpy(),
    y_true= train_state['train_targets'][best_run_index].cpu().numpy(), 
    digits=4)
)
print('Dev:',classification_report(
    y_pred=(torch.argmax(train_state['val_preds'][best_run_index],dim=1) ).cpu().long().numpy(),
    y_true= train_state['val_targets'][best_run_index].cpu().numpy(), 
    digits=4)
)

Train:               precision    recall  f1-score   support

           0     0.8956    0.9442    0.9193     40491
           1     0.7335    0.5823    0.6492     10674

    accuracy                         0.8687     51165
   macro avg     0.8145    0.7632    0.7842     51165
weighted avg     0.8617    0.8687    0.8629     51165

Dev:               precision    recall  f1-score   support

           0     0.9358    0.9358    0.9358      2243
           1     0.7563    0.7563    0.7563       591

    accuracy                         0.8984      2834
   macro avg     0.8461    0.8461    0.8461      2834
weighted avg     0.8984    0.8984    0.8984      2834



In [39]:
def sort_preds(indexes, preds):
    """Sorts the predictions in order, to reverse the effects of shuffle
    done by dataloader"""
    indexes = indexes.cpu().numpy().reshape(-1,1)
    preds = preds.cpu().numpy()
    arr_concat = np.hstack((indexes,preds)) #concat the preds and their indexes
    sort_arr = arr_concat[ arr_concat[:,0].argsort()] #sort based on the indexes
    sorted_preds = np.delete(sort_arr,0,axis=1)
    return sorted_preds

In [40]:
def get_optimal_models_v2(train_state, split):
    l = zip(train_state[f'{split}_f1s'], range(len(train_state[f'{split}_f1s'])))
    sorted_vals = sorted(l, key = lambda x:x[0], reverse=True)
    model_idxes = [i[1] for i in sorted_vals]
    
    trgts= sort_preds(train_state[f'{split}_indexes'][-1],train_state[f'{split}_targets'][-1].reshape(-1,1))
    total_preds = len(train_state[f'{split}_indexes'])
    init = np.zeros(train_state[f'{split}_preds'][-1].shape)
    max_f1 = 0
    idxes = []
    for i in model_idxes:
        temp = sort_preds(train_state[f'{split}_indexes'][i],train_state[f'{split}_preds'][i])
        temp2 = init+temp
        f1 = f1_score(
            y_pred=temp2.argmax(axis=1),
            y_true= trgts, average ='macro'
        )
        if f1 > max_f1:
            max_f1 = f1
            init = init+temp
            idxes.append(i)
    print(f'Taking preds from {idxes} | Dev f1:{f1}')
    return idxes

In [41]:
final_optimal_models = get_optimal_models_v2(train_state, 'val')
final_optimal_models

Taking preds from [0] | Dev f1:0.84607272258193


[0]

# Making preds on the given test set

In [42]:
test_df = data_df_task_b
test_df=test_df.loc[test_df['split'] == 'test']

In [43]:
test_df.sample(5)

,Unnamed: 0,id,text,average,std,label,split
55395,55395,1157022158691995648,"<s> @USER Depraved, disgusting and sick </s>",0.238162,0.200676,0,test
54283,54283,1160869002442420225,<s> it’s like looking at a fat bitch when she ...,0.218824,0.221042,0,test
56657,56657,1187557614357504000,<s> @USER Maybe fix it you dumb fucking rate </s>,0.501844,0.288658,1,test
54245,54245,1157470303670304769,<s> @USER bitch u look fine as hell can we ro...,0.269808,0.274180,0,test
55232,55232,1155831129137721345,<s> Wake up and drink your water bitch </s>,0.211810,0.209948,0,test


In [44]:
dataset = HateDataset(
    data_df = data_df_task_b,
    tokenizer = roberta_tokenizer
)

In [45]:
test_dataset = dataset
test_dataset.set_split('test')

In [46]:
print(len(test_df))
print(test_dataset._target_df.split.value_counts())

2693
test    2693
Name: split, dtype: int64


In [47]:
def evaluate_testset(model, state, dataset, split,args):
    """Returns the final layer output of our transformer model
    Puts them in the '{split}_*' keys in the state dict
    Args:
        model: A pytorch transformers model
        state: dict to store outputs
        dataset: A pytorch Dataset
        split: The split on which to evaluate the model on
        args: Arguments from namespace, etc
    Returns:
        state: all evaluated output stored in the "test" key
    """
    eval_bar = notebook.tqdm(
        desc = 'evaluation progress: ',
        total=dataset.get_num_batches(args.batch_size),
        position=0,
        leave=False,
    )
    dataset.set_split(split)
    batch_generator = generate_batches(
        dataset= dataset, batch_size= args.batch_size, shuffle=False,
        device = args.device, drop_last=False,
        pinned_memory = True, n_workers = 2, 
    )
    eval_bar.reset(
        total=dataset.get_num_batches(args.batch_size),
    )
    model.eval()
    with torch.no_grad():
        for batch_index, batch_dict in enumerate(batch_generator):
            y_pred = model(
                input_ids = batch_dict['x_data'],
                attention_mask =  batch_dict['x_attn_mask'],
            )[0]
            y_pred = y_pred.view(-1, 2)

            y_pred = y_pred.detach()
            
            state['batch_preds'].append(y_pred.cpu())
            state['batch_indexes'].append(batch_dict['x_index'].cpu())
            
            eval_bar.update()
            
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    state[f'{split}_preds'].append(
        torch.cat(state['batch_preds']).cpu()
    )
    state[f'{split}_indexes'].append(
        torch.cat(state['batch_indexes']).cpu()
    )
    
    state['batch_preds'] = []
    state['batch_indexes'] = []
    
    eval_bar.close()
    return state

In [48]:
all_model_paths = [os.path.join(args.directory,i) for i in os.listdir(args.directory) if args.model_name in i]
chosen_models = [all_model_paths[i] for i in final_optimal_models]

In [49]:
test_state = general_utils.make_train_state()
for model_path in notebook.tqdm(chosen_models, total=len(chosen_models)):
    model.load_state_dict(torch.load(model_path)['model'])
    test_state = evaluate_testset(model, test_state, test_dataset, 'test',args)

  0%|          | 0/1 [00:00<?, ?it/s]

evaluation progress:   0%|          | 0/168 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

In [50]:
test_state['test_preds'][-1].shape

torch.Size([2693, 2])

In [51]:
[test_state['test_preds'][i].size() for i in range(len(test_state['test_preds']))]

[torch.Size([2693, 2])]

In [52]:
len(test_dataset._target_df)

2693

In [53]:
torch.zeros_like(test_state['test_preds'][0]).size()

torch.Size([2693, 2])

In [54]:
ensemble_pred = torch.zeros_like(test_state['test_preds'][0])
for i in test_state['test_preds']:
    ensemble_pred += i

In [55]:
int_to_label = {0: 'TIN', 1:'UNT'}
# {'UNT': 1, 'TIN': 0}

In [56]:
y_pred = []
t = []
for i in torch.argmax(ensemble_pred, dim=1):
    y_pred.append(i)
    t.append(int_to_label[i.item()])

collections.Counter(t)

Counter({'TIN': 2109, 'UNT': 584})

In [57]:
y_true = np.array(test_df.iloc[:,5])
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      2131
           1       0.73      0.76      0.74       562

    accuracy                           0.89      2693
   macro avg       0.83      0.84      0.84      2693
weighted avg       0.89      0.89      0.89      2693



In [58]:
assert len(t) == len(test_df)

In [59]:
offeval_task_b_pred_analysis_df = pd.DataFrame(
    data={
        'id':test_df.id,
        'text':test_df.text,
        'label':test_df.label,
        # 'text':test_df.tweet,
        # 'label':t,
    }
)

In [60]:
offeval_task_b_pred_label_df = pd.DataFrame(
    data={
        'id':test_df.id,
        'label':test_df.label,
        # 'label':t,
    }
)
# offeval_task_b_label_df = pd.DataFrame(
#     data={
#         'id':test_df.id,
#         'label':test_df.label,
#         # 'label':t,
#     }
# )

In [61]:
offeval_task_b_pred_analysis_df.to_csv(
    'offeval_task_b_pred_analysis_df.csv',index=False,
)

In [62]:
offeval_task_b_pred_label_df.to_csv(
    'offeval_task_b_pred_label_df.csv', index=False, header=False,
)

In [63]:
offeval_task_b_pred_label_df.label.value_counts()

0    2131
1     562
Name: label, dtype: int64

In [64]:
offeval_task_b_pred_analysis_df.label.value_counts()

0    2131
1     562
Name: label, dtype: int64

# Making preds on the new Test Set

In [65]:
label_to_int = {'TIN': 0, 'UNT':1}
test_set = pd.read_csv(args.test_csv, sep='\t').rename(columns={'tweet':'text'})
test_label_set = pd.read_csv('/content/Intro-to-NLP-Final-Project-Fall2021/TaskB_data/test_b_labels.csv', sep=',', header=None, names=['id', 'label'])
test_set['label'] = test_label_set['label']
test_set['split'] = 'test'
test_set = test_set.replace({"label": label_to_int})
test_set['text'] = test_set['text'].map(roberta_preproc.add_special_tokens)
test_set

,id,text,label,split
0,BC0,<s> Then to top it completely the fuck off my ...,1,test
1,BC1,"<s> To which, of course, many Slovenes will re...",0,test
2,BC2,<s> @USER it means go away with your sorry ass...,0,test
3,BC3,<s> @USER @USER And he’s closer to trump’s ass...,0,test
4,BC4,<s> Looking back at old photos of me makes me ...,1,test
...,...,...,...,...
1417,BC1417,<s> @USER His name is cheating ass Myron 💀💀💀💀💀...,0,test
1418,BC1418,"<s> @USER So do I.....my neighbours, the Posti...",1,test
1419,BC1419,<s> @USER @USER That shit is badass </s>,1,test
1420,BC1420,<s> I get to work hop on here co workers aroun...,1,test


In [66]:
print(test_set[test_set['split']=='test'][test_set['label']==0].count()[0])
print(test_set[test_set['split']=='test'][test_set['label']==1].count()[0])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


850
572


In [67]:
dataset = HateDataset(
    data_df = test_set,
    tokenizer = roberta_tokenizer
)

In [68]:
test_state = general_utils.make_train_state()
for model_path in notebook.tqdm(chosen_models, total=len(chosen_models)):
    model.load_state_dict(torch.load(model_path)['model'])
    test_state = evaluate_testset(model, test_state, dataset, 'test',args)

  0%|          | 0/1 [00:00<?, ?it/s]

evaluation progress: : 0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [69]:
test_state['test_preds'][-1].shape
[test_state['test_preds'][i].size() for i in range(len(test_state['test_preds']))]
torch.zeros_like(test_state['test_preds'][0]).size()
ensemble_pred = torch.zeros_like(test_state['test_preds'][0])
for i in test_state['test_preds']:
    ensemble_pred += i
int_to_label = {0: 'TIN', 1:'UNT'}

In [70]:
y_pred = []
t = []
for i in torch.argmax(ensemble_pred, dim=1):
    y_pred.append(i)
    t.append(int_to_label[i.item()])

collections.Counter(t)

Counter({'TIN': 1256, 'UNT': 166})

In [71]:
y_true = np.array(test_set.iloc[:,2])
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.96      0.78       850
           1       0.80      0.23      0.36       572

    accuracy                           0.67      1422
   macro avg       0.73      0.60      0.57      1422
weighted avg       0.71      0.67      0.61      1422



In [72]:
with open('y_pred_distilrobeta.npy', 'wb') as f:
  np.save(f, y_pred)

## Ensemble Method

In [73]:
# Require outputs from 3 different models
# y_pred_albert = np.load('y_pred_albert.npy')
# y_pred_distilroberta = np.load('y_pred_distilrobeta.npy')
# y_pred_robeta = np.load('y_pred_robeta.npy')

In [74]:
# y_pred_ensemble = np.array([y_pred_albert, y_pred_distilroberta, y_pred_robeta])
# y_pred = []
# for i in range(len(y_pred_ensemble[0])):
#   y_pred.append(np.bincount(np.array(y_pred_ensemble)[:, i]).argmax())

# from sklearn.metrics import accuracy_score, f1_score
# print(accuracy_score(y_true, y_pred))
# print(f1_score(y_true, y_pred, average='macro'))
# print(classification_report(y_true, y_pred))